# Desafío 02

## Integrantes

- Acevedo Zain, Gaspar (acevedo.zain.gaspar@gmail.com)

## Consignas

**Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado**

Recuerden que su notebook de entrega debe poder correrse de inicio a fin sin la aparición de errores.

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro artista del dataset Songs.
- Elegir términos de interés y buscar términos más similares y menos similares.
- Realizar una reduccion de dimensionalidad a los embeddings, llevándolos a 2 dimensiones. Graficar los embeddings proyectados y seleccionar una cantidad de términos (variable MAX_WORDS) de forma tal que la visualización sea adecuada.
- Inspeccionar el grafico y buscar pequeños grupos de palabras que puedan formarse. Interpretarlos e intentar obtener conclusiones. En lo posible, acompañar los grupos de palabras con capturas (y pegarlas en celdas de texto)

## Imports

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import platform

In [4]:
import multiprocessing
try:
  from gensim.models import Word2Vec
except:
  !pip install gensim
  from gensim.models import Word2Vec

## Descarga del dataset

Se descarga el dataset provisto por el postgrado:

In [5]:
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

--2025-11-09 22:50:06--  http://songs_dataset.zip/
Resolving songs_dataset.zip (songs_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘songs_dataset.zip’
--2025-11-09 22:50:06--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2025-11-09 22:50:07--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.

## Creación del dataset

A partir de los datos descargados, se crea un dataframe con las canciones de *bob-dylan* de la siguiente manera:

In [8]:
df = pd.read_csv('songs_dataset/bob-dylan.txt', sep='/n', header=None)

/tmp/ipython-input-2133178393.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('songs_dataset/bob-dylan.txt', sep='/n', header=None)


El dataframe creado tiene un total de $5213$ documentos:

In [9]:
print(f"Cantidad de documentos: {df.shape[0]}")

Cantidad de documentos: 5213


## 1) Creación de Vectores

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro artista del dataset Songs.

### Preprocesamiento

In [19]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

In [22]:
nltk.download("punkt")
nltk.download('punkt_tab')
nltk.download("wordnet")
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [82]:
def nltk_process(text):
    # Tokenization
    nltk_tokenList = word_tokenize(text)

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    nltk_lemmaList = []
    for word in nltk_tokenList:
      replaced_word = word.replace("’", "'")
      nltk_lemmaList.append(lemmatizer.lemmatize(replaced_word))

    # Stop words
    nltk_stop_words = set(stopwords.words("english"))

    filtered_sentence = [w for w in nltk_lemmaList if w not in nltk_stop_words]

    # Filter Punctuation
    str_punctuaction = string.punctuation + "“”"
    filtered_sentence = [w for w in filtered_sentence if w not in str_punctuaction]

    return filtered_sentence

In [83]:
sentence_tokens = []

for _, row in df[:None].iterrows():
    sentence_tokens.append(nltk_process(row[0]))

### Creación de vectores (word2vec)

In [85]:
from gensim.models.callbacks import CallbackAny2Vec

In [86]:
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [93]:
w2v_model = Word2Vec(min_count=5,
                     window=2,
                     vector_size=300,
                     negative=20,
                     workers=1,
                     sg=1)

In [94]:
w2v_model.build_vocab(sentence_tokens)

In [95]:
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 5213


In [96]:
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 883


### Entrenamiento de embeddings

In [97]:
w2v_model.train(sentence_tokens,
                total_examples=w2v_model.corpus_count,
                epochs=20,
                compute_loss = True,
                callbacks=[callback()]
                )

Loss after epoch 0: 223576.984375
Loss after epoch 1: 100466.578125
Loss after epoch 2: 97382.6875
Loss after epoch 3: 98097.84375
Loss after epoch 4: 101066.78125
Loss after epoch 5: 99870.8125
Loss after epoch 6: 100150.75
Loss after epoch 7: 98466.25
Loss after epoch 8: 98171.1875
Loss after epoch 9: 87979.0
Loss after epoch 10: 83151.25
Loss after epoch 11: 82577.0
Loss after epoch 12: 81581.75
Loss after epoch 13: 80904.5
Loss after epoch 14: 79450.625
Loss after epoch 15: 78984.375
Loss after epoch 16: 78462.875
Loss after epoch 17: 78806.625
Loss after epoch 18: 76723.875
Loss after epoch 19: 77543.0


(288062, 472180)

## 2) Términos similares

- Elegir términos de interés y buscar términos más similares y menos similares.

Estos son los $5$ términos de interés que se analizarán en esta sección:
- Stone
- church
- heaven
- people
- angel

### 2.1) Términos más y menos similares a `Stone`

In [99]:
w2v_model.wv.most_similar(positive=["stone"], topn=10)

[('ya', 0.9150049686431885),
 ('They', 0.8962523341178894),
 ('say', 0.8463841080665588),
 ('Like', 0.84209805727005),
 ('head', 0.8357020020484924),
 ('rolling', 0.8324129581451416),
 ('One', 0.8166376948356628),
 ('walking', 0.8144593238830566),
 ('made', 0.8134141564369202),
 ('walked', 0.813011884689331)]

In [100]:
w2v_model.wv.most_similar(negative=["stone"], topn=10)

[('Saved', -0.042700596153736115),
 ('na', -0.2212112843990326),
 ('wake', -0.25266507267951965),
 ('Maggie', -0.27259060740470886),
 ('strengthen', -0.27856016159057617),
 ('work', -0.2801832854747772),
 ('gon', -0.28210732340812683),
 ('serve', -0.28255030512809753),
 ('fly', -0.314992219209671),
 ('somebody', -0.3244931995868683)]

### 2.2) Términos más y menos similares a `church`

In [101]:
w2v_model.wv.most_similar(positive=["church"], topn=10)

[('dry', 0.9978100061416626),
 ('Lenny', 0.9977315664291382),
 ('Brown', 0.9976727962493896),
 ('Bruce', 0.9974696040153503),
 ('shelf', 0.9973604083061218),
 ('Zanzinger', 0.9969207048416138),
 ('body', 0.9968354105949402),
 ('breeze', 0.9967702031135559),
 ('flesh', 0.9966574907302856),
 ('cast', 0.996656060218811)]

In [102]:
w2v_model.wv.most_similar(negative=["church"], topn=10)

[('Saved', -0.09006612002849579),
 ('hard', -0.5851526260375977),
 ('let', -0.593352198600769),
 ('oh', -0.5971886515617371),
 ('na', -0.6077194809913635),
 ('may', -0.6163716316223145),
 ('new', -0.6181920766830444),
 ('light', -0.6248124837875366),
 ('Tweedle-dee', -0.6253528594970703),
 ('need', -0.6387790441513062)]

### 2.3) Términos más y menos similares a `heaven`

In [106]:
w2v_model.wv.most_similar(positive=["heaven"], topn=10)

[('knockin', 0.9772059321403503),
 ('close', 0.9722503423690796),
 ('knock', 0.9657081365585327),
 ('door', 0.9561936855316162),
 ('Knock', 0.9224091172218323),
 ('around', 0.8138116598129272),
 ('His', 0.7766327857971191),
 ('eye', 0.7762775421142578),
 ('bend', 0.7739635109901428),
 ('trouble', 0.7603121995925903)]

In [107]:
w2v_model.wv.most_similar(negative=["heaven"], topn=10)

[('Saved', -0.059910088777542114),
 ('somebody', -0.24606440961360931),
 ('na', -0.2517099678516388),
 ('let', -0.2531074583530426),
 ('serve', -0.261421263217926),
 ('gon', -0.2730228006839752),
 ('work', -0.2786222994327545),
 ('Maggie', -0.2821488082408905),
 ('wake', -0.2825106680393219),
 ('strengthen', -0.2950803339481354)]

### 2.4) Términos más y menos similares a `people`

In [112]:
w2v_model.wv.most_similar(positive=["people"], topn=10)

[('n', 0.9701966643333435),
 ('seven', 0.9564288854598999),
 ('sometimes', 0.9451584815979004),
 ('ear', 0.9444698095321655),
 ('Everybody', 0.9440535306930542),
 ('year', 0.9427483081817627),
 ('round', 0.9415493011474609),
 ('letter', 0.939873456954956),
 ('turned', 0.9397830367088318),
 ('turning', 0.9394456148147583)]

In [113]:
w2v_model.wv.most_similar(negative=["people"], topn=10)

[('Saved', -0.046489011496305466),
 ('na', -0.4591003358364105),
 ('oh', -0.46463119983673096),
 ('serve', -0.4652773141860962),
 ('somebody', -0.46729397773742676),
 ('may', -0.47744324803352356),
 ('gon', -0.4876036047935486),
 ('work', -0.49128836393356323),
 ('Maggie', -0.49464061856269836),
 ('fly', -0.4950193464756012)]

### 2.5) Términos más y menos similares a `angel`

In [116]:
w2v_model.wv.most_similar(positive=["angel"], topn=10)

[('move', 0.9986103177070618),
 ('explain', 0.9980019927024841),
 ('write', 0.9979618787765503),
 ('Have', 0.9979345202445984),
 ('pay', 0.997907817363739),
 ('Will', 0.9978713393211365),
 ('Make', 0.9978373050689697),
 ('younger', 0.9977353811264038),
 ('61', 0.9977124929428101),
 ('jump', 0.997689425945282)]

In [117]:
w2v_model.wv.most_similar(negative=["angel"], topn=10)

[('Saved', -0.06909818202257156),
 ('light', -0.5999688506126404),
 ('hard', -0.6068061590194702),
 ('new', -0.6072836518287659),
 ('may', -0.61259925365448),
 ('wiggle', -0.6151480078697205),
 ('Tweedle-dee', -0.6172564625740051),
 ('oh', -0.6313378810882568),
 ('til', -0.6391488909721375),
 ('let', -0.6392382383346558)]

## 3) Reducción de dimensionalidad

- Realizar una reduccion de dimensionalidad a los embeddings, llevándolos a 2 dimensiones. Graficar los embeddings proyectados y seleccionar una cantidad de términos (variable MAX_WORDS) de forma tal que la visualización sea adecuada.

In [118]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [121]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=600
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab")

### 4) Grupos de palabras

- Inspeccionar el grafico y buscar pequeños grupos de palabras que puedan formarse.
- Interpretarlos e intentar obtener conclusiones. En lo posible, acompañar los grupos de palabras con capturas (y pegarlas en celdas de texto).